In [1]:
import sqlite3

In [2]:
class DatabaseGestion:
    def __init__(self, path='../data/olist.db'):
        self.db_path = path
        self.conn = sqlite3.connect(self.db_path)

    def do_request(self, sql_request):
        try:
            cursor = self.conn.cursor()

            # Faire la requete
            result = cursor.execute(sql_request)

            # Recuperer le resultat
            return list(result)

        except Exception as error:
            print(f"An error occurred: {error}")
            return None


In [3]:
DG = DatabaseGestion()

In [4]:
# Afficher le noms des tables dans le fichier db
DG.do_request("SELECT name FROM sqlite_master WHERE type = 'table'")

[('customers',),
 ('geoloc',),
 ('order_items',),
 ('order_pymts',),
 ('order_reviews',),
 ('orders',),
 ('products',),
 ('sellers',),
 ('translation',)]

In [5]:
# Afficher tous les champs disponible pour une table donnée
DG.do_request("PRAGMA table_info(customers);")

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'customer_id', 'TEXT', 0, None, 0),
 (2, 'customer_unique_id', 'TEXT', 0, None, 0),
 (3, 'customer_zip_code_prefix', 'BIGINT', 0, None, 0),
 (4, 'customer_city', 'TEXT', 0, None, 0),
 (5, 'customer_state', 'TEXT', 0, None, 0)]

In [6]:
# Afficher la commande la plus recente pour avoir une date de reference
DG.do_request("SELECT MAX(order_purchase_timestamp) AS date_la_plus_recente FROM orders;")

[('2018-10-17 17:30:18',)]

# Requete 1
En excluant les commandes annulées, quelles sont les commandes
récentes de moins de 3 mois que les clients ont reçues avec au moins 3
jours de retard ?

In [7]:
# Afficher les champs de la table orders
DG.do_request("PRAGMA table_info(orders);")


[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'order_id', 'TEXT', 0, None, 0),
 (2, 'customer_id', 'TEXT', 0, None, 0),
 (3, 'order_status', 'TEXT', 0, None, 0),
 (4, 'order_purchase_timestamp', 'TEXT', 0, None, 0),
 (5, 'order_approved_at', 'TEXT', 0, None, 0),
 (6, 'order_delivered_carrier_date', 'TEXT', 0, None, 0),
 (7, 'order_delivered_customer_date', 'TEXT', 0, None, 0),
 (8, 'order_estimated_delivery_date', 'TEXT', 0, None, 0)]

In [8]:
# Selection de toutes les commandes de moins de 4 mois a compté du  2018-10-17 17:30:18 et qui ne sont pas cancel
DG.do_request("""
            SELECT order_id,order_status,order_purchase_timestamp 
            FROM orders 
            WHERE order_purchase_timestamp >= date('2018-10-17 17:30:18', '-3 months') 
            AND order_purchase_timestamp <= date('2018-10-17 17:30:18') 
            AND order_status != 'canceled';
""")

[('53cdb2fc8bc7dce0b6741e2150273451', 'delivered', '2018-07-24 20:41:37'),
 ('47770eb9100c2d0c44946d9cf07ec65d', 'delivered', '2018-08-08 08:38:49'),
 ('5ff96c15d0b717ac6ad1f3d77225a350', 'delivered', '2018-07-25 17:44:10'),
 ('f3e7c359154d965827355f39d6b1fdac', 'delivered', '2018-08-09 11:44:40'),
 ('b276e4f8c0fb86bd82fce576f21713e0', 'delivered', '2018-07-29 23:34:51'),
 ('d22e9fa5731b9e30e8b27afcdc2f8563', 'delivered', '2018-08-04 23:25:30'),
 ('5820a1100976432c7968a52da59e9364', 'delivered', '2018-07-29 11:24:17'),
 ('9faeb9b2746b9d7526aef5acb08e2aa0', 'delivered', '2018-07-26 14:39:59'),
 ('f346ad4ee8f630e5e4ddaf862a34e6dd', 'delivered', '2018-08-05 13:09:48'),
 ('bd4bd0194d6d29f83b8557d4b89b572a', 'delivered', '2018-07-28 16:52:55'),
 ('6d25592267349b322799e2beb687871e', 'delivered', '2018-08-26 22:04:34'),
 ('77e9941864fc840be8e4b1ba5347c0f7', 'delivered', '2018-08-03 08:59:39'),
 ('e3447938231fb6ba2fee0231b51eca59', 'delivered', '2018-08-26 11:04:26'),
 ('a5474c0071dd5d1074e12d

In [9]:
# Selection de toutes les commandes de moins de 3 mois a compté du  2018-10-17 17:30:18 et qui ne sont pas cancel recu au minimum 3j plus tard
DG.do_request("""
SELECT 
    order_status,order_estimated_delivery_date,order_delivered_customer_date
FROM 
    orders 
WHERE 
    order_purchase_timestamp >= date('2018-10-17 17:30:18', '-3 months') 
        AND 
    order_purchase_timestamp <= date('2018-10-17 17:30:18')
        AND 
    order_status != 'canceled'
        AND
    order_estimated_delivery_date <= date(order_delivered_customer_date, '-3 days')
    
    ;""")

[('delivered', '2018-08-22 00:00:00', '2018-08-29 01:41:41'),
 ('delivered', '2018-08-23 00:00:00', '2018-09-01 18:14:42'),
 ('delivered', '2018-08-03 00:00:00', '2018-08-14 04:04:40'),
 ('delivered', '2018-08-09 00:00:00', '2018-08-13 20:11:47'),
 ('delivered', '2018-08-17 00:00:00', '2018-08-21 00:11:52'),
 ('delivered', '2018-08-10 00:00:00', '2018-08-14 17:21:32'),
 ('delivered', '2018-08-10 00:00:00', '2018-08-14 17:51:20'),
 ('delivered', '2018-08-14 00:00:00', '2018-08-22 14:15:51'),
 ('delivered', '2018-08-13 00:00:00', '2018-08-20 15:55:42'),
 ('delivered', '2018-08-13 00:00:00', '2018-08-24 17:03:18'),
 ('delivered', '2018-08-17 00:00:00', '2018-08-21 18:19:01'),
 ('delivered', '2018-08-10 00:00:00', '2018-08-15 17:58:41'),
 ('delivered', '2018-08-08 00:00:00', '2018-08-13 16:54:31'),
 ('delivered', '2018-08-14 00:00:00', '2018-08-21 11:28:49'),
 ('delivered', '2018-08-09 00:00:00', '2018-08-13 16:12:25'),
 ('delivered', '2018-08-23 00:00:00', '2018-08-28 01:07:29'),
 ('deliv

# Requete 2
Qui sont les vendeurs ayant généré un chiffre d'affaires de plus de 100
000 Real sur des commandes livrées via Olist ?

In [10]:
# Afficher tous les champs disponible pour une table donnée
DG.do_request("PRAGMA table_info(order_items);")

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'order_id', 'TEXT', 0, None, 0),
 (2, 'order_item_id', 'BIGINT', 0, None, 0),
 (3, 'product_id', 'TEXT', 0, None, 0),
 (4, 'seller_id', 'TEXT', 0, None, 0),
 (5, 'shipping_limit_date', 'TEXT', 0, None, 0),
 (6, 'price', 'FLOAT', 0, None, 0),
 (7, 'freight_value', 'FLOAT', 0, None, 0)]

In [11]:
DG.do_request("""
SELECT 
    seller_id, 
    SUM(price) AS total_revenue
FROM 
    order_items
JOIN 
    orders ON order_items.order_id = orders.order_id  
WHERE 
    orders.order_status = 'delivered' 
GROUP BY 
    seller_id
HAVING 
    total_revenue > 100000;
""")

[('1025f0e2d44d7041d6cf58b6550e0bfa', 138208.56),
 ('1f50f920176fa81dab994f9023523100', 106655.70999999999),
 ('46dc3b2cc0980fb8ec44634e21d2718e', 122811.38),
 ('4869f7a5dfa277a7dca6462dcf3b52b2', 226987.93),
 ('4a3ca9315b744ce9f8e9374361493884', 196882.12),
 ('53243585a1d6dc2643021fd1853d8905', 217940.44),
 ('5dceca129747e92ff8ef7a997dc4f8ca', 111126.73),
 ('620c87c171fb2a6dd6e8bb4dec959fc6', 112461.5),
 ('6560211a19b47992c3666cc44a7e94c0', 120702.83),
 ('7a67c85e85bb2ce8582c35f2203ad736', 139658.69),
 ('7c67e1448b00f6e969d365cea6b010ab', 186570.05),
 ('7d13fca15225358621be4086e1eb0964', 112436.18000000001),
 ('7e93a43ef30c4f03f38b393420bc753a', 165981.49),
 ('955fee9216a65b617aa5c0531780ce60', 131836.71),
 ('cc419e0650a3c5ba77189a1882b7556a', 101090.95999999999),
 ('da8622b14eb17ae2831f4ac5b9dab84a', 159816.87),
 ('fa1c13f2614d7b5c4749cbc52fecda94', 190917.14)]

# Requete 3 
Qui sont les nouveaux vendeurs (moins de 3 mois d'ancienneté) qui
sont déjà très engagés avec la plateforme (ayant déjà vendu plus de 30
produits) ?

In [12]:
# Afficher tous les champs disponible pour une table donnée
DG.do_request("PRAGMA table_info(sellers);")

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'seller_id', 'TEXT', 0, None, 0),
 (2, 'seller_zip_code_prefix', 'BIGINT', 0, None, 0),
 (3, 'seller_city', 'TEXT', 0, None, 0),
 (4, 'seller_state', 'TEXT', 0, None, 0)]

Pour identifier les nouveaux vendeurs (moins de 3 mois d'ancienneté) qui ont déjà vendu plus de 30 produits, nous devons examiner les dates de vente et le nombre de produits vendus. Nous allons joindre les tables order_items et   sellers et ensuite filtrer les résultats en fonction de la date d'inscription des vendeurs et du nombre de produits vendus.

In [13]:
DG.do_request("""
SELECT 
    s.seller_id,
    COUNT(oi.product_id) AS total_products_sold
FROM 
    order_items oi
JOIN 
    sellers s ON oi.seller_id = s.seller_id
JOIN 
    orders o ON oi.order_id = o.order_id
WHERE 
    o.order_purchase_timestamp >= date('2018-10-17 17:30:18', '-3 months')
GROUP BY 
    s.seller_id
HAVING 
    COUNT(oi.product_id) > 30;
""")

[('0241d4d5d36f10f80c644447315af0bd', 50),
 ('04308b1ee57b6625f47df1d56f00eedf', 33),
 ('06a2c3af7b3aee5d69171b0e14f0ee87', 81),
 ('1025f0e2d44d7041d6cf58b6550e0bfa', 247),
 ('16090f2ca825584b5a147ab24aa30c86', 84),
 ('1835b56ce799e6a4dc4eddc053f04066', 71),
 ('1900267e848ceeba8fa32d80c1a5f5a8', 68),
 ('1da3aeb70d7989d1e6d9b0e887f97c23', 35),
 ('1f50f920176fa81dab994f9023523100', 69),
 ('213b25e6f54661939f11710a6fddb871', 51),
 ('240b9776d844d37535668549a396af32', 36),
 ('289cdb325fb7e7f891c38608bf9e0962', 70),
 ('391fc6631aebcf3004804e51b40bcf1e', 33),
 ('3d871de0142ce09b7081e2b9d1733cb1', 46),
 ('4869f7a5dfa277a7dca6462dcf3b52b2', 127),
 ('4a3ca9315b744ce9f8e9374361493884', 99),
 ('4b9750c8ad28220fe6702d4ecb7c898f', 65),
 ('4d6d651bd7684af3fffabd5f08d12e5a', 31),
 ('53243585a1d6dc2643021fd1853d8905', 112),
 ('53e4c6e0f4312d4d2107a8c9cddf45cd', 41),
 ('5656537e588803a555b8eb41f07a944b', 38),
 ('6061155addc1e54b4cfb51c1c2a32ad8', 31),
 ('612170e34b97004b3ba37eae81836b4c', 62),
 ('620c8

# Requete 4
Quels sont les 5 codes postaux, enregistrant plus de 30
reviews, avec le pire review score moyen sur les 12 derniers mois ?

In [14]:
# Afficher tous les champs disponible pour une table donnée
DG.do_request("PRAGMA table_info(order_reviews);")

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'review_id', 'TEXT', 0, None, 0),
 (2, 'order_id', 'TEXT', 0, None, 0),
 (3, 'review_score', 'BIGINT', 0, None, 0),
 (4, 'review_comment_title', 'TEXT', 0, None, 0),
 (5, 'review_comment_message', 'TEXT', 0, None, 0),
 (6, 'review_creation_date', 'TEXT', 0, None, 0),
 (7, 'review_answer_timestamp', 'TEXT', 0, None, 0)]

Pour obtenir les 5 codes postaux avec plus de 30 reviews et le pire score moyen de reviews sur les 12 derniers mois, nous devons joindre les tables order_reviews, orders, et customers

In [15]:
DG.do_request("""
SELECT 
    c.customer_zip_code_prefix,
    AVG(orv.review_score) AS avg_review_score,
    COUNT(orv.review_id) AS review_count
FROM 
    order_reviews orv
JOIN 
    orders o ON orv.order_id = o.order_id
JOIN 
    customers c ON o.customer_id = c.customer_id
WHERE 
    orv.review_creation_date >= date('2018-10-17 17:30:18', '-12 months')
GROUP BY 
    c.customer_zip_code_prefix
HAVING 
    COUNT(orv.review_id) > 30
ORDER BY 
    avg_review_score ASC
LIMIT 5;


""")

[(22753, 2.8333333333333335, 48),
 (22770, 3.1842105263157894, 38),
 (22793, 3.247311827956989, 93),
 (13056, 3.2903225806451615, 31),
 (13295, 3.3125, 32)]